In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

In [ ]:
# !which chromedriver

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/nadiarichards/.wdm/drivers/chromedriver/mac64/88.0.4324.96]


In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8867/nasas-mars-helicopter-reports-in/" target="_self">NASA's Mars Helicopter Reports In </a></div>

In [6]:
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Mars Helicopter Reports In "

In [7]:
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The technology demonstration has phoned home from where it is attached to the belly of NASA’s Perseverance rover. '

In [ ]:
browser.quit()

In [8]:
url_images = 'https://www.jpl.nasa.gov/images?search=&category=Mars'
browser.visit(url_images)

In [ ]:
html = browser.html
img_soup = soup(html, 'html.parser')

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

sidebar = soup.find('ul', class_='nav-list')

categories = sidebar.find_all('li')

category_list = []
url_list = []
book_url_list = []
for category in categories:
    title = category.text.strip()
    category_list.append(title)
    book_url = category.find('a')['href']
    url_list.append(book_url)

book_url_list = ['http://books.toscrape.com/' + url for url in url_list]

titles_and_urls = zip(category_list, book_url_list)

try:
    for title_url in titles_and_urls:
        browser.links.find_by_partial_text('next').click()
except ElementDoesNotExist:
    print("Scraping Complete")

In [ ]:
img_soup.prettify()

In [ ]:
img_elem.find("div", class_='content_title')

In [ ]:
img_url_rel = img_soup.select_one('img.BaseImage object-contain')
img_url_rel

In [ ]:
.get("src")